In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tutorial.settings")
import django
django.setup()

# Working with Serializers

http://www.django-rest-framework.org/tutorial/1-serialization/#working-with-serializers

In [2]:
from snippets.models import Snippet
for s in Snippet.objects.all():
    s.delete()

snippet = Snippet(code='foo = "bar"\n')
snippet.save()

snippet2 = Snippet(code='print "hello, world"\n')
snippet2.save()

In [3]:
Snippet.objects.all()

<QuerySet [<Snippet: Snippet object>, <Snippet: Snippet object>]>

In [4]:
from snippets.serializers import SnippetSerializer
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser

In [5]:
serializer = SnippetSerializer(snippet2)

In [6]:
print(serializer.data)

{'style': 'friendly', 'code': u'print "hello, world"\n', 'language': 'python', 'title': u'', 'linenos': False, 'id': 14}


In [7]:
content = JSONRenderer().render(serializer.data)
print(content)

{"id":14,"title":"","code":"print \"hello, world\"\n","linenos":false,"language":"python","style":"friendly"}


In [8]:
import io

In [9]:
stream = io.BytesIO(content)
data = JSONParser().parse(stream)

serializer2 = SnippetSerializer(data=data)

In [10]:
serializer2.is_valid()

True

In [11]:
serializer2.validated_data

OrderedDict([(u'title', u''),
             (u'code', u'print "hello, world"'),
             (u'linenos', False),
             (u'language', 'python'),
             (u'style', 'friendly')])

In [12]:
print(Snippet.objects.count())
serializer2.save()
print(Snippet.objects.count())

2
3


In [13]:
print('\n'.join(
    line[:100] + ('...' if len(line) > 100 else '') 
    for line 
    in repr(serializer).splitlines())
) # maybe reprs with lots of choices should be shortened...

SnippetSerializer(<Snippet: Snippet object>):
    id = IntegerField(label='ID', read_only=True)
    title = CharField(allow_blank=True, max_length=100, required=False)
    code = CharField(style={'base_template': 'textarea.html'})
    linenos = BooleanField(required=False)
    language = ChoiceField(choices=[('abap', 'ABAP'), ('abnf', 'ABNF'), ('ada', 'Ada'), ('adl', 'ADL...
    style = ChoiceField(choices=[('algol', 'algol'), ('algol_nu', 'algol_nu'), ('autumn', 'autumn'),...


# Running the Server

We'll do it from the notebook because we're super cheeky.

In [14]:
import subprocess
import sys
import time

In [15]:
DJPORT = 8805
dj = subprocess.Popen(
    [sys.executable, 'manage.py', 'runserver', str(DJPORT)], 
    universal_newlines=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
time.sleep(0.5)

In [16]:
import requests
server = 'http://127.0.0.1:{:d}'.format(DJPORT)
print(server)

http://127.0.0.1:8805


In [17]:
response = requests.get(server + '/snippets')
print(response)
response.json()

<Response [200]>


[{u'code': u'foo = "bar"\n',
  u'id': 13,
  u'language': u'python',
  u'linenos': False,
  u'style': u'friendly',
  u'title': u''},
 {u'code': u'print "hello, world"\n',
  u'id': 14,
  u'language': u'python',
  u'linenos': False,
  u'style': u'friendly',
  u'title': u''},
 {u'code': u'print "hello, world"',
  u'id': 15,
  u'language': u'python',
  u'linenos': False,
  u'style': u'friendly',
  u'title': u''}]

In [18]:
SOME_ID = serializer.data['id']

response = requests.get(server + '/snippets/{}'.format(SOME_ID))
print(response)
response.json()

<Response [200]>


{u'code': u'print "hello, world"\n',
 u'id': 14,
 u'language': u'python',
 u'linenos': False,
 u'style': u'friendly',
 u'title': u''}

In [19]:
dj.terminate()